## DFT TESTBENCH

This notebook takes two inputs (real and imaginary) and gived the real and imaginary parts of the DFT outputs using AXI-STREAM. It is then compared with software version of FFT

In [1]:
from pynq import Overlay
import numpy as np
from pynq import allocate
from pynq.lib import dma
from scipy.linalg import dft
import matplotlib.pyplot as plt

In [2]:
ol=Overlay('dft.bit')
#ol.ip_dict

In [3]:
# get dma objects
dma_0=ol.axi_dma_0
dma_0_send = ol.axi_dma_0.sendchannel
dma_0_recv = ol.axi_dma_0.recvchannel

dma_1 = ol.axi_dma_1
dma_1_send = ol.axi_dma_1.sendchannel
dma_1_recv = ol.axi_dma_1.recvchannel

# get access to overlay
dft_ip = ol.dft_0
dft_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0)
}

In [4]:
NUM_SAMPLES = 8

real_error=np.zeros(NUM_SAMPLES)
imag_error=np.zeros(NUM_SAMPLES)
#ind=np.arange(NUM_SAMPLES)
real_rmse=np.zeros(NUM_SAMPLES)
imag_rmse=np.zeros(NUM_SAMPLES)

In [ ]:
# get access to overlay
dft_ip.write(0x0,0x1)
dft_ip.register_map

in_r  = allocate(shape=(NUM_SAMPLES,), dtype=np.float32) 
in_i  = allocate(shape=(NUM_SAMPLES,), dtype=np.float32)           
out_r = allocate(shape=(NUM_SAMPLES,), dtype=np.float32) 
out_i = allocate(shape=(NUM_SAMPLES,), dtype=np.float32)

a = [i for i in range(NUM_SAMPLES)]
a = np.float32(a)
a = np.cos(a)
real = a.real                # Change input real and imaginary value here
img  = a.imag
np.copyto(in_r, real)
np.copyto(in_i, img)
for i in range(NUM_SAMPLES):
     print((in_r[i], in_i[i]))


(1.0, 0.0)
(0.5403023, 0.0)
(-0.41614684, 0.0)
(-0.9899925, 0.0)
(-0.6536436, 0.0)
(0.2836622, 0.0)
(0.96017027, 0.0)
(0.75390226, 0.0)


In [ ]:
#Send and recv data 
dma_1_recv.transfer(out_i)
dma_1_send.transfer(in_i)

dma_1_send.wait()
dma_1_recv.wait() 

dma_0_recv.transfer(out_r)
dma_0_send.transfer(in_r)

dma_0_send.wait()
dma_0_recv.wait()


## Verifying Functionality 

In [ ]:
golden_op=np.fft.fft(a)

for i in range(NUM_SAMPLES):
    real_error[i]="{0:.6f}".format(abs(out_r[i]-golden_op.real[i]))
    imag_error[i]="{0:.6f}".format(abs(out_i[i]-golden_op.imag[i]))

In [ ]:
sum_sq_real=0
sum_sq_imag=0
for i in range(NUM_SAMPLES):
    sum_sq_real =sum_sq_real+(real_error[i]*real_error[i])
    real_rmse = np.sqrt(sum_sq_real / (i+1))
    sum_sq_imag =sum_sq_imag+(imag_error[i]*imag_error[i])
    imag_rmse = np.sqrt(sum_sq_imag / (i+1))
print("Real Part RMSE: ", real_rmse, "Imaginary Part RMSE:", imag_rmse)    
if real_rmse<0.001 and imag_rmse<0.001:
    print("PASS")
else:
    print("FAIL")

## Displaying Error and Output

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1,2,1)
plt.bar(ind,real_error)
plt.title("Real Part Error")
plt.xlabel("Index")
plt.ylabel("Error")
#plt.xticks(ind)
plt.tight_layout()

plt.subplot(1,2,2)
plt.bar(ind,imag_error)
plt.title("Imaginary Part Error")
plt.xlabel("Index")
plt.ylabel("Error")
#plt.xticks(ind)
plt.tight_layout()

In [ ]:
freq=np.fft.fftfreq(NUMSAMPLES)

plt.figure(figsize=(7, 4))
plt.subplot(1,2,1)
plt.plot(freq,out_r,label='real')
plt.plot(freq,out_i,label='imag')
plt.title("512-DFT")
plt.xlabel("Frequency")
plt.ylabel("DFT real and imaginary data")
plt.legend()
plt.tight_layout()
plt.subplot(1,2,2)
plt.plot(freq,golden_op.real,label='real')
plt.plot(freq,golden_op.imag,label='imag')
plt.title("512-FFT -Numpy")
plt.xlabel("Frequency")
plt.ylabel("FFT real and imaginary data")
plt.legend()
plt.tight_layout()
plt.show()